In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

idx  = pd.IndexSlice
% matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st

In [89]:
##### 1 #### CREATION D'UNE DATAFRAME DE BASE AVEC INDEX NUMERIQUE PAR DEFAUT

my_df = pd.DataFrame([[0.1,10,20,220],[1,11,21,221],[2,12,22,222],[3,13,23,223],\
                       [0.1,10,20,220],[1,11,21,221],[2,12,22,222],[3,13,23,223]],\
                      index = [['GROWTH','GROWTH','GROWTH','GROWTH','EDUCATION','EDUCATION','EDUCATION','EDUCATION'],\
                               ['ind1','ind1','ind1','ind1','ind2','ind2','ind2','ind2'],\
                               ['indicateur1','indicateur1','indicateur1','indicateur1',\
                                'indicateur2','indicateur2','indicateur2','indicateur2'],\
                               ['Europe','Europe','Africa','Africa']*2,\
                               ['FRA','GER','ALG','TUN']*2,\
                               ['France','Germany','Algeria','Tunisia']*2],\
                              columns = ['1990', '1991', '1992', '1993'])

my_df3 = pd.DataFrame( {'Topic' : ['GROWTH','GROWTH','GROWTH','GROWTH','EDUCATION','EDUCATION','EDUCATION','EDUCATION'],\
                        'Indicator Code' : ['ind1','ind1','ind1','ind1','ind2','ind2','ind2','ind2'],\
                        'Indicator Name' : ['indicateur1','indicateur1','indicateur1','indicateur1',\
                                 'indicateur2','indicateur2','indicateur2','indicateur2'],\
                        'Region' : ['Europe','Europe','Africa','Africa']*2,\
                        'Country Name' : ['France','Germany','Algeria','Tunisia']*2,\
                        'Country Code' : ['FRA','GER','ALG','TUN']*2,\
                       })
my_df.reset_index(inplace = False)[my_df.columns]
my_df4 = pd.concat([my_df3, my_df.reset_index(inplace = False)[my_df.columns]], axis =1, ignore_index = False)

##### 2 #### TRANSFORMATION DE COLONNES EN INDEXS
my_df4.set_index(['Topic', 'Indicator Code','Indicator Name', 'Region', 'Country Name', 'Country Code'], inplace = True)
##### 3 #### PERMUTATION DES INDEX DE LIGNES ET DE COLONNES DE LA DATAFRAME
my_df5 = my_df4.copy(deep = True)
my_df5.columns = pd.MultiIndex.from_product([my_df4.columns, ['val']], names = ['Year', 'nom'])
my_df5.columns = my_df5.columns.swaplevel(0, 1)
my_df5 = my_df5.unstack(['Topic', 'Indicator Code', 'Indicator Name']).stack('Year')
#my_df5.index = my_df5.index.swaplevel(0, 1)
my_df5.sort_index(inplace=True)
my_df5.columns = my_df5.columns.droplevel()
my_df5.T.dropna(how = 'all', inplace = False, axis = 0).T
my_df5.dropna(axis = 1, inplace = True)
my_df5.head()

Topic                                   EDUCATION      GROWTH
Indicator Code                               ind2        ind1
Indicator Name                        indicateur2 indicateur1
Region Country Name Country Code Year                        
Africa Algeria      ALG          1990         2.0         2.0
                                 1991        12.0        12.0
                                 1992        22.0        22.0
                                 1993       222.0       222.0
       Tunisia      TUN          1990         3.0         3.0

In [90]:
my_df6 = my_df5.copy(deep = True)

In [91]:
my_df6.loc[('Africa', 'Algeria', 'ALG', '1992'),('GROWTH', 'ind1')] = np.nan
my_df6

Topic                                   EDUCATION      GROWTH
Indicator Code                               ind2        ind1
Indicator Name                        indicateur2 indicateur1
Region Country Name Country Code Year                        
Africa Algeria      ALG          1990         2.0         2.0
                                 1991        12.0        12.0
                                 1992        22.0         NaN
                                 1993       222.0       222.0
       Tunisia      TUN          1990         3.0         3.0
                                 1991        13.0        13.0
                                 1992        23.0        23.0
                                 1993       223.0       223.0
Europe France       FRA          1990         0.1         0.1
                                 1991        10.0        10.0
                                 1992        20.0        20.0
                                 1993       220.0       220.0
       Germany      GER          1990         1.0         1.0
                                 1991        11.0        11.0
                                 1992        21.0        21.0
                                 1993       221.0       221.0

In [120]:
my_df7 = my_df6.copy(deep = True)
#[(c1,c2,c3)][my_df6[(c1,c2,c3)].where(~np.isnan(my_df6[(c1,c2,c3)]))] = "null !"
my_df7.T

Region                                   Africa                                               Europe                                              
Country Name                            Algeria                    Tunisia                    France                    Germany                   
Country Code                                ALG                        TUN                       FRA                        GER                   
Year                                       1990  1991  1992   1993    1990  1991  1992   1993   1990  1991  1992   1993    1990  1991  1992   1993
Topic     Indicator Code Indicator Name                                                                                                           
EDUCATION ind2           indicateur2        2.0  12.0  22.0  222.0     3.0  13.0  23.0  223.0    0.1  10.0  20.0  220.0     1.0  11.0  21.0  221.0
GROWTH    ind1           indicateur1        2.0  12.0   NaN  222.0     3.0  13.0  23.0  223.0    0.1  10.0  20.0  220.0     1.0  11.0  21.0  221.0

In [128]:
#remplacer les Nan par 2022202, remplacer les non null par le double
# ('EDUCATION','ind2','indicateur2')  ('GROWTH','ind1','indicateur1')
tupl_ind = ('GROWTH','ind1','indicateur1')
dfc = (my_df7[tupl_ind] - my_df7[tupl_ind].mean())/my_df7[tupl_ind].std(ddof = 0)
dfc

Region  Country Name  Country Code  Year
Africa  Algeria       ALG           1990   -0.692878
                                    1991   -0.585964
                                    1992         NaN
                                    1993    1.659244
        Tunisia       TUN           1990   -0.682187
                                    1991   -0.575272
                                    1992   -0.468357
                                    1993    1.669936
Europe  France        FRA           1990   -0.713192
                                    1991   -0.607347
                                    1992   -0.500432
                                    1993    1.637861
        Germany       GER           1990   -0.703570
                                    1991   -0.596655
                                    1992   -0.489740
                                    1993    1.648553
Name: (GROWTH, ind1, indicateur1), dtype: float64

In [78]:
# df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
#                     'B': [5, 6, 7, 8, 9],
#                     'C': ['a', 'b', 'c', 'd', 'e']})
df.replace(np.nan, 5)

Topic                                   EDUCATION      GROWTH
Indicator Code                               ind2        ind1
Indicator Name                        indicateur2 indicateur1
Region Country Name Country Code Year                        
Africa Algeria      ALG          1990       400.0       400.0
                                 1991      2400.0      2400.0
                                 1992      4400.0         5.0
                                 1993         5.0         5.0
       Tunisia      TUN          1990       600.0       600.0
                                 1991      2600.0      2600.0
                                 1992      4600.0      4600.0
                                 1993         5.0         5.0
Europe France       FRA          1990         0.0         0.0
                                 1991      2000.0      2000.0
                                 1992      4000.0      4000.0
                                 1993         5.0         5.0
       Germany      GER          1990       200.0       200.0
                                 1991      2200.0      2200.0
                                 1992      4200.0      4200.0
                                 1993         5.0         5.0

In [41]:
for (c1,c2,c3) in my_df6.columns :
    if np.isnan(my_df6[(c1,c2,c3)]):
        my_df6_z[(c1,c2,c3)] = np.nan
    else :
        my_df6_z[(c1,c2,c3)] = (my_df6[(c1,c2,c3)] - my_df6[(c1,c2,c3)].mean()) / my_df6[(c1,c2,c3)].std(ddof=0)
my_df6_z.head()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [28]:
# somme de toutes les valeurs années par pays et pour un indicateur choisi
my_df6.groupby(level="Country Name").sum()["indicateur1"] 

Country Name
Algeria    258.0
France     250.0
Germany    254.0
Tunisia    262.0
Name: indicateur1, dtype: float64

In [ ]:
i=0
for year, new_df in my_df5.groupby(level=3):
    if i == 0 :
        coucou =new_df
coucou

for index in my_df5.index.get_level_values('Region') :
    print(index)
for col in my_df5.columns.get_level_values('Topic'):
    print(col)

# ##### 4 #### INTERROGATION DE LA DATAFRAME
# #slices...